In [1]:
import corr
import torch_lddmm

import numpy as np
import torch

In [2]:
frames = corr.load_example_experiment()
template_idx = corr.find_highest_correlation(frames)
template = frames[template_idx]

In [3]:
torch.from_numpy(template)

tensor([[  56.,   64., 1064.,  ...,  526.,   57.,   44.],
        [1796.,   68.,   68.,  ...,   71.,   62.,   54.],
        [  74.,  567., 1147.,  ...,   58.,   66.,  127.],
        ...,
        [  66.,   64., 3370.,  ...,   64.,   58.,  274.],
        [  71.,   57.,   72.,  ...,   51.,   55.,   53.],
        [  88., 1515.,   63.,  ...,   64., 1157.,   58.]])

In [5]:
lddmm = torch_lddmm.LDDMM(template=template, target=frames[0], a=8, epsilon=1.0, sigma=10.0, sigmaR=10.0, dx=[0.1,0.1,0.1]) # create object
lddmm.run() # run registration with these settings
(vt0,vt1,vt2) = lddmm.outputTransforms() # output LDDMM transforms
(phi0,phi1,phi2) = lddmm.computeThisDisplacement() # output resultant displacement field
deformed_template = lddmm.outputDeformedTemplate() # output deformed template as numpy array


Current parameters:
>    a               = 8 (smoothing kernel, a*(pixel_size))
>    p               = 2 (smoothing kernel power, p*2)
>    niter           = 100 (number of iterations)
>    epsilon         = 1.0 (gradient descent step size)
>    epsilonL        = 1e-07 (gradient descent step size, affine)
>    epsilonT        = 2e-05 (gradient descent step size, translation)
>    minbeta         = 0.0001 (smallest multiple of epsilon)
>    sigma           = 10.0 (matching term coefficient (0.5/sigma**2))
>    sigmaR          = 10.0 (regularization term coefficient (0.5/sigmaR**2))
>    nt              = 5 (number of time steps in velocity field)
>    do_lddmm        = 1 (perform LDDMM step, 0 = no, 1 = yes)
>    do_affine       = 0 (interleave linear registration: 0 = no, 1 = affine, 2 = rigid, 3 = rigid + scale)
>    checkaffinestep = 0 (evaluate linear matching energy: 0 = no, 1 = yes)
>    im_norm_ms      = 0 (normalize image by mean and std: 0 = no, 1 = yes)
>    gpu_number      =

/Users/admin/Documents/Studium/10_Semester/bimap/bimap/src/torch_lddmm.py:202: SyntaxWarning: invalid escape sequence '\/'
  print('WARNING: template file name is not set. Use LDDMM.setParams(\'template\',filename\/array).\n')
/Users/admin/Documents/Studium/10_Semester/bimap/bimap/src/torch_lddmm.py:216: SyntaxWarning: invalid escape sequence '\/'
  print('WARNING: target file name is not set. Use LDDMM.setParams(\'target\',filename\/array).\n')


AssertionError: Torch not compiled with CUDA enabled